<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ---------------------------------------------------------- Лабораторная работа №1

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings('ignore')

# --- Parameters ---
url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/dataset.csv"
combined_data_path = "inheritance_combined.csv"

numeric_features = [
    'debts', 'funeral_expenses', 'bequests', 'wife', 'husband', 'son', 'daughter',
    'father', 'mother', 'brother', 'sister', 'grandchild'
]

# --- Data Loading ---
try:
    df = pd.read_csv(url)
    print("Данные успешно загружены.")
except Exception as e:
    print(f"Ошибка загрузки данных: {e}")
    raise

print("Столбцы:", df.columns.tolist())
print("\nИнформация о данных:")
print(df.info())
print("\nСтатистика по пропущенным значениям:")
print(df[numeric_features].isnull().sum())

# --- Data Cleaning ---
df_clean = df[numeric_features].dropna()
for col in ['debts', 'funeral_expenses', 'bequests']:
    df_clean = df_clean[df_clean[col] > 0]
df_clean.reset_index(drop=True, inplace=True)

print(f"\nРазмер очищенных данных: {df_clean.shape}")

# --- Visualize Distributions ---
plt.figure(figsize=(15, 8))
for i, col in enumerate(['debts', 'funeral_expenses', 'bequests'], 1):
    plt.subplot(1, 3, i)
    sns.histplot(df_clean[col], bins=30, kde=True)
    plt.title(f'Распределение {col}')
plt.tight_layout()
plt.show()

# --- Correlation Heatmap ---
plt.figure(figsize=(10, 8))
sns.heatmap(df_clean.corr(), annot=True, cmap='coolwarm')
plt.title('Корреляционная матрица')
plt.show()

# --- Scaling ---
minmax_scaler = MinMaxScaler()
df_minmax = pd.DataFrame(
    minmax_scaler.fit_transform(df_clean),
    columns=[f"{col}_minmax" for col in numeric_features]
)

standard_scaler = StandardScaler()
df_standard = pd.DataFrame(
    standard_scaler.fit_transform(df_clean),
    columns=[f"{col}_standard" for col in numeric_features]
)

# --- Combine all data into one DataFrame ---
df_combined = pd.concat([df_clean, df_minmax, df_standard], axis=1)

# --- Save combined DataFrame ---
df_combined.to_csv(combined_data_path, index=False)
print(f"\nВсе данные (очищенные и масштабированные) сохранены в {combined_data_path}")

# --- Show Data Samples ---
print("\nПервые строки объединённых данных:\n", df_combined.head())

# --- Save scalers for future use ---
import joblib
joblib.dump(minmax_scaler, 'minmax_scaler.pkl')
joblib.dump(standard_scaler, 'standard_scaler.pkl')


Ошибка загрузки данных: No columns to parse from file


EmptyDataError: No columns to parse from file